In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
import WormholeRayTracer as wrmhole
import WormholeGraphics as wg
import Symplectic_DNeg as Smpl

In [3]:
path = os.getcwd()

In [10]:
Par = [0.8/1.42953, 8.6, 4.3] # M, rho, a parameters wormhole

initial_q = np.array([12, np.pi, np.pi/2])

In [11]:
print(initial_q)
Grid_dimension = '3D'
mode = 0
Motion1, Grid1, CM1 = wrmhole.Simulate_DNeg(Smpl.Sympl_DNeg, Par, 0.02, 1500, initial_q, 20**2, 20**2, Grid_dimension, mode, wg.Grid_constr_3D_Sph)

[12.          3.14159265  1.57079633]
138.87587475776672


In [12]:
Cosyst =  ' Sph '
cv2.imwrite(os.path.join(path, "Pictures/Image "+Grid_dimension+"Gr"+Cosyst+"DNeg Sympl"+str(Par)+" "+str(initial_q)+".png"),
            255*wg.Make_Pict_RGBP(Motion1[-1, 1], Grid1))

True